<a href="https://colab.research.google.com/github/Tech-pooja/CAD/blob/main/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [408]:
# introduction

# background info

# motivation - computational cost
# objective

# metgods and models - working flowchart

# 20-30 pages


#Pandas is commonly used for data manipulation and analysis, while numpy is used for numerical computations.
import pandas as pd
import numpy as np

#for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

#ColumnTransformer is used to apply different transformations to different columns of the dataset.
#StandardScaler and MinMaxScaler are used for feature scaling.
#OneHotEncoder is used for converting categorical variables into numerical representation.
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

#train_test_split is used to split the dataset into training and testing subsets.
#StratifiedKFold is a cross-validation method that ensures each fold has the same proportion of class labels as the whole dataset.
#cross_validate is used to perform cross-validation and evaluate the model's performance.
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

from sklearn.ensemble import RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

#for evaluating the performance of the model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

#These lines import the warnings module and suppress all warnings that may occur during the execution of the code.
import warnings
warnings.filterwarnings('ignore')

In [409]:
import io
#df = pd.read_csv(io.BytesIO(uploaded['CAD.csv']))
df = pd.read_csv("/content/CAD.csv")

In [410]:
df.head()

,Age,Weight,Length,Sex,BMI,DM,HTN,Current Smoker,EX-Smoker,FH,...,K,Na,WBC,Lymph,Neut,PLT,EF-TTE,Region RWMA,VHD,Cath
0,53,90,175,Male,29.387755,0,1,1,0,0,...,4.7,141,5700,39,52,261,50,0,N,Cad
1,67,70,157,Fmale,28.398718,0,1,0,0,0,...,4.7,156,7700,38,55,165,40,4,N,Cad
2,54,54,164,Male,20.077335,0,0,1,0,0,...,4.7,139,7400,38,60,230,40,2,mild,Cad
3,66,67,158,Fmale,26.838648,0,1,0,0,0,...,4.4,142,13000,18,72,742,55,0,Severe,Normal
4,50,87,153,Fmale,37.165193,0,1,0,0,0,...,4.0,140,9200,55,39,274,50,0,Severe,Normal


In [411]:
# Check for missing values

total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum() / df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent*100], axis=1, keys=["Total", "Percent(%)"])
missing_data

,Total,Percent(%)
Age,0,0.0
HDL,0,0.0
LowTH Ang,0,0.0
Q Wave,0,0.0
St Elevation,0,0.0
St Depression,0,0.0
Tinversion,0,0.0
LVH,0,0.0
Poor R Progression,0,0.0
FBS,0,0.0


In [412]:
# Check for duplicates

duplicates = df[df.duplicated()]
print("Total Duplicates rows observed:", duplicates.shape[0])
print("Dropping duplicates")

# Dropping duplicates
print("Shape before dropping duplicates: ", df.shape[0])
df.drop_duplicates(inplace=True)
print("Shae after dropping duplicates: ", df.shape[0])

Total Duplicates rows observed: 0
Dropping duplicates
Shape before dropping duplicates:  303
Shae after dropping duplicates:  303


In [413]:
# Check for unique values

# Numerical variables:
num_cols = ['Age','Weight', 'Length','BMI', 'BP', 'PR', 'FBS', 'CR', 'TG', 'LDL', 'HDL', 'BUN', 'ESR', 'HB', 'K', 'Na', 'WBC', 'Lymph', 'Neut', 'PLT', 'EF-TTE']

# Categorical variables:
cat_cols = ['Sex', 'DM', 'HTN', 'Current Smoker', 'EX-Smoker', 'FH', 'Obesity', 'CRF', 'CVA', 'Airway disease', 'Thyroid Disease', 'CHF', 'DLP', 'Edema', 'Weak Peripheral Pulse', 'Lung rales', 'Systolic Murmur', 'Diastolic Murmur', 'Typical Chest Pain', 'Dyspnea', 'Atypical', 'Nonanginal', 'Exertional CP', 'LowTH Ang', 'Q Wave', 'St Elevation', 'St Depression', 'Tinversion', 'LVH', 'Poor R Progression', 'Cath']

# Ordinal variables
ord_cols = ['Function Class', "Region RWMA", "VHD"]

print(f"[Unique Values in {len(cat_cols)} Categorical Variables]\n")

for cat_col in cat_cols:
    print("* {} : {} Unique Values =>".format(cat_col, df[cat_col].nunique()), df[cat_col].unique())


[Unique Values in 31 Categorical Variables]

* Sex : 2 Unique Values => ['Male' 'Fmale']
* DM : 2 Unique Values => [0 1]
* HTN : 2 Unique Values => [1 0]
* Current Smoker : 2 Unique Values => [1 0]
* EX-Smoker : 2 Unique Values => [0 1]
* FH : 2 Unique Values => [0 1]
* Obesity : 2 Unique Values => ['Y' 'N']
* CRF : 2 Unique Values => ['N' 'Y']
* CVA : 2 Unique Values => ['N' 'Y']
* Airway disease : 2 Unique Values => ['N' 'Y']
* Thyroid Disease : 2 Unique Values => ['N' 'Y']
* CHF : 2 Unique Values => ['N' 'Y']
* DLP : 2 Unique Values => ['Y' 'N']
* Edema : 2 Unique Values => [0 1]
* Weak Peripheral Pulse : 2 Unique Values => ['N' 'Y']
* Lung rales : 2 Unique Values => ['N' 'Y']
* Systolic Murmur : 2 Unique Values => ['N' 'Y']
* Diastolic Murmur : 2 Unique Values => ['N' 'Y']
* Typical Chest Pain : 2 Unique Values => [0 1]
* Dyspnea : 2 Unique Values => ['N' 'Y']
* Atypical : 2 Unique Values => ['N' 'Y']
* Nonanginal : 2 Unique Values => ['N' 'Y']
* Exertional CP : 1 Unique Values => 

In [414]:
# Data shape

print(f"Dataset : {df.shape[0]} rows X {df.shape[1]} columns")

Dataset : 303 rows X 55 columns


In [415]:
# data stats

df[num_cols].describe()

,Age,Weight,Length,BMI,BP,PR,FBS,CR,TG,LDL,...,BUN,ESR,HB,K,Na,WBC,Lymph,Neut,PLT,EF-TTE
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,...,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,58.897690,73.831683,164.716172,27.248339,129.554455,75.141914,119.184818,1.055611,150.343234,104.643564,...,17.501650,19.462046,13.153465,4.230693,140.996700,7562.046205,32.399340,60.148515,221.488449,47.231023
std,10.392278,11.987358,9.327661,4.098865,18.938105,8.911801,52.079653,0.264296,97.959451,35.396688,...,6.956761,15.936475,1.610452,0.458202,3.807885,2413.739323,9.972592,10.182493,60.796199,8.927194
min,30.000000,48.000000,140.000000,18.115413,90.000000,50.000000,62.000000,0.500000,37.000000,18.000000,...,6.000000,1.000000,8.900000,3.000000,128.000000,3700.000000,7.000000,32.000000,25.000000,15.000000
25%,51.000000,65.000000,158.000000,24.514380,120.000000,70.000000,88.500000,0.900000,90.000000,80.000000,...,13.000000,9.000000,12.200000,3.900000,139.000000,5800.000000,26.000000,52.500000,183.500000,45.000000
50%,58.000000,74.000000,165.000000,26.775510,130.000000,70.000000,98.000000,1.000000,122.000000,100.000000,...,16.000000,15.000000,13.200000,4.200000,141.000000,7100.000000,32.000000,60.000000,210.000000,50.000000
75%,66.000000,81.000000,171.000000,29.411765,140.000000,80.000000,130.000000,1.200000,177.000000,122.000000,...,20.000000,26.000000,14.200000,4.500000,143.000000,8800.000000,39.000000,67.000000,250.000000,55.000000
max,86.000000,120.000000,188.000000,40.900658,190.000000,110.000000,400.000000,2.200000,1050.000000,232.000000,...,52.000000,90.000000,17.600000,6.600000,156.000000,18000.000000,60.000000,89.000000,742.000000,60.000000


In [416]:
#Distribution of continuous features are not uniform, and has a certain skewness. Especially, feature TG (max value = 1050),
#FBS (max value = 400), and HDL (max value = 111) and other features has some outliers.
#We will check outliers by plotting boxplot in the next part.

vhd = {"N": 0, "mild": 1, "Moderate": 2, "Severe": 3}
sex = {"Male": "Male", "Fmale": "Female"}

df['VHD'] = df['VHD'].map(vhd)
df['Sex'] = df['Sex'].map(sex)

df.replace('N', 0, inplace=True)
df.replace('Y', 1, inplace=True)

df.head()

,Age,Weight,Length,Sex,BMI,DM,HTN,Current Smoker,EX-Smoker,FH,...,K,Na,WBC,Lymph,Neut,PLT,EF-TTE,Region RWMA,VHD,Cath
0,53,90,175,Male,29.387755,0,1,1,0,0,...,4.7,141,5700,39,52,261,50,0,0,Cad
1,67,70,157,Female,28.398718,0,1,0,0,0,...,4.7,156,7700,38,55,165,40,4,0,Cad
2,54,54,164,Male,20.077335,0,0,1,0,0,...,4.7,139,7400,38,60,230,40,2,1,Cad
3,66,67,158,Female,26.838648,0,1,0,0,0,...,4.4,142,13000,18,72,742,55,0,3,Normal
4,50,87,153,Female,37.165193,0,1,0,0,0,...,4.0,140,9200,55,39,274,50,0,3,Normal


In [417]:
X = df.drop("Cath", axis=1)
y = df['Cath']

map_label = {"Cad":1, "Normal":0}
y = y.map(map_label)

In [418]:
# Numerical variables:
num_cols = ['Age','Weight', 'Length','BMI', 'BP', 'PR', 'FBS', 'CR', 'TG', 'LDL', 'HDL', 'BUN', 'ESR', 'HB', 'K', 'Na', 'WBC', 'Lymph', 'Neut', 'PLT', 'EF-TTE']

# Categorical variables:
cat_cols = ['Sex', 'DM', 'HTN', 'Current Smoker', 'EX-Smoker', 'FH', 'Obesity', 'CRF', 'CVA', 'Airway disease', 'Thyroid Disease', 'CHF', 'DLP', 'Edema', 'Weak Peripheral Pulse', 'Lung rales', 'Systolic Murmur', 'Diastolic Murmur', 'Typical Chest Pain', 'Dyspnea', 'Atypical', 'Nonanginal', 'Exertional CP', 'LowTH Ang', 'Q Wave', 'St Elevation', 'St Depression', 'Tinversion', 'LVH', 'Poor R Progression', 'Cath']
cat_cols.remove('Cath')

# Ordinal variables
ord_cols = ['Function Class', "Region RWMA", "VHD"]

In [419]:
preprocessor = ColumnTransformer(transformers = [('OHE', OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first', dtype=np.int64), cat_cols),
                                                 ('Scaler', StandardScaler(), num_cols)],
                                 remainder = 'passthrough',verbose_feature_names_out=False).set_output(transform = 'pandas')
X_prep = preprocessor.fit_transform(X)

In [420]:
X_prep.head()

,Sex_Male,DM_1,HTN_1,Current Smoker_1,EX-Smoker_1,FH_1,Obesity_1,CRF_1,CVA_1,Airway disease_1,...,K,Na,WBC,Lymph,Neut,PLT,EF-TTE,Function Class,Region RWMA,VHD
0,1,0,1,1,0,0,1,0,0,0,...,1.025931,0.000868,-0.772712,0.662975,-0.801571,0.650977,0.310686,0,0,0
1,0,0,1,0,0,0,1,0,0,0,...,1.025931,3.946579,0.057248,0.562534,-0.506461,-0.930681,-0.811339,0,4,0
2,1,0,0,1,0,0,0,0,0,0,...,1.025931,-0.525227,-0.067246,0.562534,-0.014609,0.140233,-0.811339,0,2,1
3,0,0,1,0,0,0,1,0,0,0,...,0.370114,0.263916,2.256644,-1.446280,1.165833,8.575744,0.871699,3,0,3
4,0,0,1,0,0,0,1,0,0,0,...,-0.504308,-0.262179,0.679719,2.270027,-2.080384,0.865160,0.310686,2,0,3


In [421]:
X_prep.columns

Index(['Sex_Male', 'DM_1', 'HTN_1', 'Current Smoker_1', 'EX-Smoker_1', 'FH_1',
       'Obesity_1', 'CRF_1', 'CVA_1', 'Airway disease_1', 'Thyroid Disease_1',
       'CHF_1', 'DLP_1', 'Edema_1', 'Weak Peripheral Pulse_1', 'Lung rales_1',
       'Systolic Murmur_1', 'Diastolic Murmur_1', 'Typical Chest Pain_1',
       'Dyspnea_1', 'Atypical_1', 'Nonanginal_1', 'LowTH Ang_1', 'Q Wave_1',
       'St Elevation_1', 'St Depression_1', 'Tinversion_1', 'LVH_1',
       'Poor R Progression_1', 'Age', 'Weight', 'Length', 'BMI', 'BP', 'PR',
       'FBS', 'CR', 'TG', 'LDL', 'HDL', 'BUN', 'ESR', 'HB', 'K', 'Na', 'WBC',
       'Lymph', 'Neut', 'PLT', 'EF-TTE', 'Function Class', 'Region RWMA',
       'VHD'],
      dtype='object')

In [422]:
X_train, X_test, y_train, y_test = train_test_split(X_prep, y, test_size=0.1, stratify=y,random_state = 1)

print("Training Data Shape : ", X_train.shape, y_train.shape)
print("Test Data Shape : ", X_test.shape, y_test.shape)

Training Data Shape :  (272, 53) (272,)
Test Data Shape :  (31, 53) (31,)


In [423]:
# K nearest neighbours Classifier
knn_classifier = KNeighborsClassifier(n_neighbors = 10)
knn_classifier.fit(X_train, y_train)
knn_predictions = knn_classifier.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_predictions)
knn_precision = precision_score(y_test, knn_predictions)
knn_recall = recall_score(y_test, knn_predictions)
knn_conf_matrix = confusion_matrix(y_test, knn_predictions)
print("KNN Accuracy:", knn_accuracy)
print("KNN Precision:", knn_precision)
print("KNN Recall:", knn_recall)
print("KNN Confusion Matrix:")
print(knn_conf_matrix)

KNN Accuracy: 0.9032258064516129
KNN Precision: 0.88
KNN Recall: 1.0
KNN Confusion Matrix:
[[ 6  3]
 [ 0 22]]


In [424]:

svm_classifier = SVC(kernel='linear', C = 0.1, random_state = 1)
svm_classifier.fit(X_train, y_train)
svm_predictions = svm_classifier.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_predictions)
svm_precision = precision_score(y_test, svm_predictions)
svm_recall = recall_score(y_test, svm_predictions)
svm_conf_matrix = confusion_matrix(y_test, svm_predictions)

print("SVM Accuracy:", svm_accuracy)
print("SVM Precision:", svm_precision)
print("SVM Recall:", svm_recall)
print("SVM Confusion Matrix:")
print(svm_conf_matrix)

SVM Accuracy: 0.9032258064516129
SVM Precision: 0.88
SVM Recall: 1.0
SVM Confusion Matrix:
[[ 6  3]
 [ 0 22]]


In [425]:
rf_classifier = RandomForestClassifier(n_estimators = 10, random_state = 1)
rf_classifier.fit(X_train, y_train)
rf_predictions = rf_classifier.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_precision = precision_score(y_test, rf_predictions)
rf_recall = recall_score(y_test, rf_predictions)
rf_conf_matrix = confusion_matrix(y_test, rf_predictions)

print("RF Accuracy:", rf_accuracy)
print("RF Precision:", rf_precision)
print("RF Recall:", rf_recall)
print("RF Confusion Matrix:")
print(rf_conf_matrix)

X_train.columns

RF Accuracy: 0.9032258064516129
RF Precision: 0.88
RF Recall: 1.0
RF Confusion Matrix:
[[ 6  3]
 [ 0 22]]


Index(['Sex_Male', 'DM_1', 'HTN_1', 'Current Smoker_1', 'EX-Smoker_1', 'FH_1',
       'Obesity_1', 'CRF_1', 'CVA_1', 'Airway disease_1', 'Thyroid Disease_1',
       'CHF_1', 'DLP_1', 'Edema_1', 'Weak Peripheral Pulse_1', 'Lung rales_1',
       'Systolic Murmur_1', 'Diastolic Murmur_1', 'Typical Chest Pain_1',
       'Dyspnea_1', 'Atypical_1', 'Nonanginal_1', 'LowTH Ang_1', 'Q Wave_1',
       'St Elevation_1', 'St Depression_1', 'Tinversion_1', 'LVH_1',
       'Poor R Progression_1', 'Age', 'Weight', 'Length', 'BMI', 'BP', 'PR',
       'FBS', 'CR', 'TG', 'LDL', 'HDL', 'BUN', 'ESR', 'HB', 'K', 'Na', 'WBC',
       'Lymph', 'Neut', 'PLT', 'EF-TTE', 'Function Class', 'Region RWMA',
       'VHD'],
      dtype='object')

In [426]:
from sklearn.feature_selection import SelectKBest, f_classif

# Initialize SelectKBest with the f_classif scoring function
selector = SelectKBest(score_func=f_classif, k=25)  # Ensure at least 5 features are selected

# Fit the selector to the training data and transform the data
X_train_selected = selector.fit_transform(X_train, y_train)

# Get the indices of selected features
selected_feature_indices = selector.get_support(indices=True)

# Get the names of selected features
selected_features = X_train.columns[selected_feature_indices].tolist()

# Print the selected features
selected_features





['DM_1',
 'HTN_1',
 'CRF_1',
 'Diastolic Murmur_1',
 'Typical Chest Pain_1',
 'Dyspnea_1',
 'Atypical_1',
 'Nonanginal_1',
 'Q Wave_1',
 'St Elevation_1',
 'St Depression_1',
 'Tinversion_1',
 'Poor R Progression_1',
 'Age',
 'BP',
 'PR',
 'FBS',
 'TG',
 'BUN',
 'ESR',
 'K',
 'Lymph',
 'Neut',
 'EF-TTE',
 'Region RWMA']

In [427]:
X_new = X_prep[selected_features]
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.1, stratify=y,random_state = 1)
# X_new.head()
X_new.columns

Index(['DM_1', 'HTN_1', 'CRF_1', 'Diastolic Murmur_1', 'Typical Chest Pain_1',
       'Dyspnea_1', 'Atypical_1', 'Nonanginal_1', 'Q Wave_1', 'St Elevation_1',
       'St Depression_1', 'Tinversion_1', 'Poor R Progression_1', 'Age', 'BP',
       'PR', 'FBS', 'TG', 'BUN', 'ESR', 'K', 'Lymph', 'Neut', 'EF-TTE',
       'Region RWMA'],
      dtype='object')

In [432]:
knn_classifier = KNeighborsClassifier(n_neighbors =4)
knn_classifier.fit(X_train, y_train)
knn_predictions = knn_classifier.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_predictions)
knn_precision = precision_score(y_test, knn_predictions)
knn_recall = recall_score(y_test, knn_predictions)
knn_conf_matrix = confusion_matrix(y_test, knn_predictions)
print("KNN Accuracy:", knn_accuracy)
print("KNN Precision:", knn_precision)
print("KNN Recall:", knn_recall)
print("KNN Confusion Matrix:")
print(knn_conf_matrix)

KNN Accuracy: 0.9032258064516129
KNN Precision: 0.88
KNN Recall: 1.0
KNN Confusion Matrix:
[[ 6  3]
 [ 0 22]]


In [429]:
svm_classifier = SVC(kernel='linear', C = 1)
svm_classifier.fit(X_train, y_train)
svm_predictions = svm_classifier.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_predictions)
svm_precision = precision_score(y_test, svm_predictions)
svm_recall = recall_score(y_test, svm_predictions)
svm_conf_matrix = confusion_matrix(y_test, svm_predictions)

print("SVM Accuracy:", svm_accuracy)
print("SVM Precision:", svm_precision)
print("SVM Recall:", svm_recall)
print("SVM Confusion Matrix:")
print(svm_conf_matrix)

SVM Accuracy: 0.9032258064516129
SVM Precision: 0.88
SVM Recall: 1.0
SVM Confusion Matrix:
[[ 6  3]
 [ 0 22]]


In [435]:
rf_classifier = RandomForestClassifier(n_estimators = 300)
rf_classifier.fit(X_train, y_train)
rf_predictions = rf_classifier.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_precision = precision_score(y_test, rf_predictions)
rf_recall = recall_score(y_test, rf_predictions)
rf_conf_matrix = confusion_matrix(y_test, rf_predictions)

print("RF Accuracy:", rf_accuracy)
print("RF Precision:", rf_precision)
print("RF Recall:", rf_recall)
print("RF Confusion Matrix:")
print(rf_conf_matrix)


RF Accuracy: 0.9032258064516129
RF Precision: 0.88
RF Recall: 1.0
RF Confusion Matrix:
[[ 6  3]
 [ 0 22]]


In [431]:
from sklearn.linear_model import LogisticRegression

# Initialize the Logistic Regression classifier
logistic_classifier = LogisticRegression()

# Train the classifier on the training data
logistic_classifier.fit(X_train, y_train)

# Make predictions on the test data
logistic_predictions = logistic_classifier.predict(X_test)

# Calculate evaluation metrics
logistic_accuracy = accuracy_score(y_test, logistic_predictions)
logistic_precision = precision_score(y_test, logistic_predictions)
logistic_recall = recall_score(y_test, logistic_predictions)
logistic_conf_matrix = confusion_matrix(y_test, logistic_predictions)

# Print evaluation metrics
print("Logistic Regression Accuracy:", logistic_accuracy)
print("Logistic Regression Precision:", logistic_precision)
print("Logistic Regression Recall:", logistic_recall)
print("Logistic Regression Confusion Matrix:")
print(logistic_conf_matrix)


Logistic Regression Accuracy: 0.9032258064516129
Logistic Regression Precision: 0.88
Logistic Regression Recall: 1.0
Logistic Regression Confusion Matrix:
[[ 6  3]
 [ 0 22]]
